In [1]:
import json 
import glob 
import xml.dom.minidom as minidom
import os
import cv2
import glob
import xml
import random

In [2]:
face_color = (100, 200, 5)
limb_color = (0, 255, 0)
other_color = (200, 200, 50)
kp_color = (0, 0, 255)

segm_colors = [face_color] * 5 + [other_color] * 2 + [limb_color] * 4 + [other_color] * 4

In [3]:
def random_color():
    levels = range(32,256,32)
    return tuple(random.choice(levels) for _ in range(3))

In [4]:
def draw_keypoint(im, keypoints):
    '''
        order: 
        0-4 (face): left eye, right eye, nose, left earbase, right earbase
        5-16 (limbs):   L_F_elbow, R_F_elbow, L_B_elbow, R_B_elbow
                        L_F_knee, R_F_knee, L_B_knee, R_B_knee
                        L_F_paw, R_F_paw, L_B_paw, R_B_paw
        17-19 (others): throat, withers, tailbase
    '''
    segmts = [  (0,1), (0,2), (1,2), (0,3), (1,4),
                (2,17), (18,19),
                (5,9), (6,10), (7,11), (8,12),
                (9,13), (10,14), (11,15), (12,16)]


    for i in range(len(segmts)):
        segm = segmts[i]
        kp1 = keypoints[segm[0]]
        kp2 = keypoints[segm[1]]
        if kp1[2] == 0 or kp2[2] == 0:
            continue

        cv2.line(im, tuple(kp1[:2]), tuple(kp2[:2]), segm_colors[i], thickness=2)

    for kp in keypoints:
        if kp[2] == 0:
            continue
        cv2.circle(im, tuple(kp[:2]), radius=4, color=kp_color, thickness=-1)

    return im

In [5]:
def draw_bbox(im, xmin, ymin, xmax, ymax, color):
    cv2.rectangle(im, (xmin, ymin), (xmax, ymax), color, thickness=2)
    return im

In [15]:
def visualize_json():
    output_dir = "visualization_animalpose"
    os.makedirs(output_dir, exist_ok=True)
    anno_dict = json.load(open("keypoints.json"))
    image_map = anno_dict["images"]
    annotations = anno_dict["annotations"]
    im_dict = dict()
    for anno in annotations:
        imagename = image_map[str(anno["image_id"])]
        bbox = anno["bbox"]
        keypoints = anno["keypoints"]
        image_path = os.path.join("images", imagename)
        if not imagename in im_dict:
            im = cv2.imread(image_path)
        else:
            im = im_dict[imagename]
        
        xmin, ymin, xmax, ymax = bbox 
        im = draw_bbox(im, xmin, ymin, xmax, ymax, random_color())
        im = draw_keypoint(im, keypoints)
        save_path = os.path.join(output_dir, imagename)
        cv2.imwrite(save_path, im)
        im_dict[imagename] = im
        print(keypoints)

In [16]:
visualize_json()

[[193, 216, 1], [160, 217, 1], [174, 261, 1], [204, 186, 1], [152, 182, 1], [0, 0, 0], [0, 0, 0], [273, 168, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [266, 225, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [190, 145, 1], [351, 238, 1]]
[[330, 84, 1], [245, 91, 1], [290, 137, 1], [340, 71, 1], [225, 91, 1], [325, 223, 1], [168, 196, 1], [0, 0, 0], [94, 134, 1], [325, 253, 1], [166, 256, 1], [0, 0, 0], [53, 188, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [41, 209, 1], [285, 202, 1], [0, 0, 0], [0, 0, 0]]
[[275, 102, 1], [247, 105, 1], [269, 156, 1], [271, 71, 1], [241, 78, 1], [0, 0, 0], [195, 130, 1], [0, 0, 0], [82, 135, 1], [250, 212, 1], [203, 203, 1], [108, 195, 1], [67, 191, 1], [270, 258, 1], [202, 256, 1], [120, 242, 1], [62, 251, 1], [247, 128, 1], [217, 58, 1], [83, 71, 1]]
[[0, 0, 0], [327, 161, 1], [350, 192, 1], [333, 141, 1], [323, 145, 1], [0, 0, 0], [252, 216, 1], [0, 0, 0], [156, 143, 1], [231, 275, 1], [271, 268, 1], [145, 177, 1], [130, 163, 1], [230, 29

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [283, 187, 1], [0, 0, 0], [322, 190, 1], [0, 0, 0], [0, 0, 0], [293, 190, 1], [322, 191, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [317, 204, 1], [0, 0, 0], [0, 0, 0], [279, 162, 1], [329, 163, 1]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [234, 178, 1], [0, 0, 0], [243, 192, 1], [254, 193, 1], [223, 193, 0], [235, 194, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [240, 203, 1], [0, 0, 0], [0, 0, 0], [237, 168, 1]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [199, 153, 1], [0, 0, 0], [0, 0, 0], [186, 185, 1], [197, 184, 1], [0, 0, 0], [0, 0, 0], [186, 198, 1], [193, 195, 1], [0, 0, 0], [0, 0, 0], [186, 206, 1], [194, 205, 1], [0, 0, 0], [192, 153, 1], [191, 174, 1]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [166, 147, 1], [176, 145, 1], [0, 0, 0], [0, 0, 0], [157, 187, 1], [168, 187, 1], [0, 0, 0], [0, 0, 0], [155, 189, 1], [166, 191, 1], [0, 0, 0], [0, 0, 0], [152, 202, 1], [160, 206, 1], [0, 0, 0], [170, 153, 1],

[[121, 112, 1], [82, 114, 1], [103, 133, 1], [136, 78, 1], [63, 81, 1], [171, 223, 1], [56, 219, 1], [0, 0, 0], [0, 0, 0], [152, 240, 1], [43, 223, 1], [0, 0, 0], [0, 0, 0], [130, 281, 1], [8, 257, 1], [250, 256, 1], [220, 274, 1], [95, 162, 1], [180, 85, 1], [370, 202, 1]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [167, 197, 1], [336, 189, 1], [0, 0, 0], [0, 0, 0], [179, 300, 1], [323, 302, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [240, 123, 1]]
[[285, 149, 1], [284, 144, 1], [272, 142, 1], [291, 149, 1], [0, 0, 0], [301, 184, 1], [303, 182, 1], [333, 182, 1], [329, 176, 1], [300, 201, 1], [0, 0, 0], [333, 198, 1], [0, 0, 0], [302, 209, 1], [307, 211, 1], [323, 211, 1], [333, 207, 1], [283, 167, 1], [300, 154, 1], [345, 161, 1]]
[[242, 97, 1], [220, 97, 1], [229, 113, 1], [256, 96, 1], [209, 92, 1], [261, 165, 1], [224, 164, 1], [308, 142, 1], [315, 139, 0], [255, 186, 1], [239, 187, 1], [0, 0, 0], [279, 178, 1], [256, 2

[[0, 0, 0], [164, 151, 1], [171, 160, 1], [0, 0, 0], [159, 143, 1], [0, 0, 0], [131, 172, 1], [0, 0, 0], [70, 169, 1], [122, 192, 1], [130, 193, 1], [58, 192, 1], [66, 189, 1], [125, 207, 1], [132, 209, 1], [61, 211, 1], [74, 211, 1], [149, 165, 1], [122, 140, 1], [60, 142, 1]]
[[323, 210, 1], [320, 211, 1], [323, 213, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [294, 208, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [311, 200, 1], [286, 202, 1]]
[[0, 0, 0], [147, 282, 1], [172, 290, 1], [142, 272, 1], [137, 276, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [147, 312, 1], [123, 292, 1], [38, 305, 1]]
[[182, 203, 1], [178, 203, 1], [179, 206, 1], [183, 201, 1], [174, 202, 1], [175, 226, 1], [171, 227, 1], [0, 0, 0], [143, 229, 1], [176, 230, 1], [172, 233, 1], [0, 0, 0], [140, 235, 1], [174, 234, 1], [174, 233, 1], [0, 0, 0],

[[290, 190, 1], [174, 147, 1], [179, 302, 1], [364, 137, 1], [137, 94, 1], [374, 463, 1], [222, 424, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [226, 451, 1], [482, 488, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [221, 371, 1], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [220, 240, 1], [0, 0, 0], [219, 210, 1], [0, 0, 0], [175, 198, 1], [0, 0, 0], [112, 186, 1], [181, 214, 1], [169, 214, 1], [96, 212, 1], [110, 213, 1], [186, 237, 1], [0, 0, 0], [93, 237, 1], [109, 237, 1], [199, 198, 1], [199, 170, 1], [96, 157, 1]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [302, 177, 1], [279, 187, 1], [0, 0, 0], [243, 185, 1], [0, 0, 0], [200, 217, 1], [274, 221, 1], [248, 213, 1], [0, 0, 0], [195, 247, 1], [279, 258, 1], [306, 207, 1], [251, 240, 1], [229, 251, 1], [0, 0, 0], [275, 152, 1], [175, 233, 1]]
[[0, 0, 0], [217, 199, 1], [389, 331, 1], [0, 0, 0], [137, 131, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 

...

[[85, 46, 1], [44, 50, 1], [62, 81, 1], [97, 48, 1], [33, 46, 1], [145, 184, 1], [0, 0, 0], [259, 165, 1], [0, 0, 0], [136, 225, 1], [100, 228, 1], [264, 204, 1], [0, 0, 0], [139, 270, 1], [98, 261, 1], [269, 248, 1], [211, 232, 1], [72, 138, 1], [137, 60, 1], [0, 0, 0]]
[[346, 131, 1], [249, 133, 1], [298, 188, 1], [370, 108, 1], [228, 116, 1], [405, 395, 1], [0, 0, 0], [663, 369, 1], [0, 0, 0], [378, 504, 1], [320, 501, 1], [814, 448, 1], [730, 472, 1], [381, 629, 1], [332, 615, 1], [0, 0, 0], [0, 0, 0], [300, 316, 1], [408, 180, 1], [745, 189, 1]]
[[254, 25, 1], [228, 24, 1], [249, 42, 1], [0, 0, 0], [219, 37, 1], [0, 0, 0], [196, 132, 1], [0, 0, 0], [93, 127, 1], [184, 159, 1], [200, 165, 1], [0, 0, 0], [72, 161, 1], [200, 205, 1], [168, 200, 1], [0, 0, 0], [70, 206, 1], [240, 77, 1], [186, 40, 1], [63, 51, 1]]
[[228, 73, 1], [197, 73, 1], [212, 93, 1], [234, 64, 1], [192, 65, 1], [0, 0, 0], [164, 164, 1], [0, 0, 0], [80, 141, 1], [195, 195, 1], [176, 207, 1], [0, 0, 0], [64, 161, 

[[387, 291, 1], [0, 0, 0], [352, 316, 1], [404, 285, 1], [0, 0, 0], [463, 387, 1], [0, 0, 0], [555, 402, 1], [0, 0, 0], [449, 421, 1], [0, 0, 0], [579, 425, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [404, 345, 1], [456, 322, 1], [582, 345, 1]]
[[0, 0, 0], [426, 49, 1], [485, 79, 1], [479, 43, 1], [403, 52, 1], [0, 0, 0], [323, 226, 1], [0, 0, 0], [74, 235, 1], [346, 303, 1], [329, 311, 1], [71, 299, 1], [25, 283, 1], [339, 370, 1], [318, 377, 1], [82, 353, 1], [25, 373, 1], [419, 155, 1], [320, 86, 1], [40, 116, 1]]
[[334, 115, 1], [290, 115, 1], [320, 146, 1], [344, 103, 1], [276, 103, 1], [0, 0, 0], [223, 225, 1], [0, 0, 0], [110, 228, 1], [289, 283, 1], [246, 277, 1], [134, 264, 1], [90, 262, 1], [296, 317, 1], [242, 317, 1], [0, 0, 0], [91, 308, 1], [302, 193, 1], [0, 0, 0], [0, 0, 0]]
[[498, 83, 1], [408, 90, 1], [458, 136, 1], [506, 59, 1], [396, 66, 1], [0, 0, 0], [333, 288, 1], [0, 0, 0], [131, 242, 1], [445, 369, 1], [355, 376, 1], [193, 324, 1], [117, 334, 1]

[[293, 101, 1], [234, 99, 1], [268, 149, 1], [299, 110, 1], [220, 103, 1], [0, 0, 0], [150, 265, 1], [0, 0, 0], [0, 0, 0], [228, 359, 1], [174, 351, 1], [0, 0, 0], [92, 318, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [252, 218, 1], [189, 135, 1], [0, 0, 0]]
[[426, 129, 1], [396, 130, 1], [413, 151, 1], [432, 122, 1], [390, 121, 1], [0, 0, 0], [364, 241, 1], [0, 0, 0], [244, 229, 1], [396, 278, 1], [380, 282, 1], [0, 0, 0], [224, 269, 1], [396, 307, 1], [384, 309, 1], [241, 309, 1], [222, 315, 1], [410, 195, 1], [361, 143, 1], [217, 165, 1]]
[[266, 173, 1], [225, 161, 1], [234, 203, 1], [285, 157, 1], [224, 140, 1], [323, 261, 1], [0, 0, 0], [387, 277, 1], [0, 0, 0], [280, 281, 1], [0, 0, 0], [429, 323, 1], [0, 0, 0], [333, 298, 1], [0, 0, 0], [368, 311, 1], [0, 0, 0], [260, 243, 1], [320, 185, 1], [462, 270, 1]]
[[305, 388, 1], [226, 394, 1], [263, 457, 1], [322, 354, 1], [208, 353, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [148, 371, 1], [310, 419, 1], [215, 431, 1], [0, 0, 0], [159, 

[[438, 113, 1], [382, 115, 1], [414, 147, 1], [447, 101, 1], [366, 105, 1], [0, 0, 0], [354, 299, 1], [0, 0, 0], [176, 291, 1], [0, 0, 0], [390, 345, 1], [0, 0, 0], [128, 323, 1], [0, 0, 0], [399, 395, 1], [101, 379, 1], [116, 387, 1], [406, 197, 1], [333, 163, 1], [156, 183, 1]]
[[62, 40, 1], [49, 40, 1], [56, 55, 1], [67, 38, 1], [43, 38, 1], [84, 77, 1], [0, 0, 0], [138, 75, 1], [0, 0, 0], [80, 102, 1], [95, 98, 1], [152, 92, 1], [138, 93, 1], [78, 121, 1], [100, 116, 1], [150, 116, 1], [133, 113, 1], [61, 71, 1], [78, 43, 1], [154, 43, 1]]
[[299, 169, 1], [256, 178, 1], [283, 206, 1], [306, 147, 1], [243, 158, 1], [0, 0, 0], [257, 245, 1], [0, 0, 0], [188, 181, 1], [287, 265, 1], [278, 290, 1], [0, 0, 0], [179, 208, 1], [291, 282, 1], [288, 324, 1], [0, 0, 0], [181, 233, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[574, 43, 1], [508, 61, 1], [550, 98, 1], [0, 0, 0], [477, 55, 1], [0, 0, 0], [392, 235, 1], [0, 0, 0], [202, 257, 1], [372, 295, 1], [404, 295, 1], [106, 296, 1], [175, 310, 1

[[656, 113, 1], [0, 0, 0], [580, 181, 1], [683, 93, 1], [565, 82, 1], [0, 0, 0], [526, 314, 1], [0, 0, 0], [264, 329, 1], [514, 439, 1], [572, 443, 1], [264, 445, 1], [204, 432, 1], [487, 529, 1], [573, 559, 1], [357, 539, 1], [212, 548, 1], [646, 260, 1], [520, 129, 1], [174, 160, 1]]
[[250, 91, 1], [166, 91, 1], [214, 148, 1], [259, 76, 1], [152, 70, 1], [309, 374, 1], [0, 0, 0], [508, 317, 1], [0, 0, 0], [272, 493, 1], [188, 479, 1], [532, 402, 1], [434, 407, 1], [290, 573, 1], [195, 567, 1], [546, 510, 1], [422, 512, 1], [198, 280, 1], [302, 159, 1], [0, 0, 0]]
[[245, 93, 1], [210, 94, 1], [232, 115, 1], [248, 85, 1], [202, 87, 1], [0, 0, 0], [159, 140, 1], [0, 0, 0], [86, 125, 1], [201, 175, 1], [166, 178, 1], [106, 151, 1], [81, 155, 1], [198, 194, 1], [0, 0, 0], [111, 173, 1], [82, 178, 1], [204, 145, 1], [0, 0, 0], [0, 0, 0]]
[[99, 97, 1], [72, 99, 1], [86, 119, 1], [114, 81, 1], [59, 83, 1], [121, 202, 1], [58, 205, 1], [153, 183, 1], [0, 0, 0], [106, 240, 1], [69, 236, 1], [1